# Number of threads speedup analysis

This notebook analyzes how much of an impact using a greater number of threads has in the optimized solutions' cost and energy consumption.

**TODO:**

[ ] add CLI option to fix number of threads in optimizer search

[ ] get energy consumption

In [1]:
USER = "st76i1_1"
GROUP = "st76i1"
PASSWORD = input()

**Change this:**

In [2]:
import sys
sys.path.append('/home/lucca/propagation-code-optimization') # change this

### Generate runs

In [ ]:
# make upload
!sshpass -p {PASSWORD} \
    scp -r ../../optimizer/ {USER}@chome.metz.supelec.fr:/usr/users/{GROUP}/{USER}/

In [ ]:
# run algorithm on cluster
runs = [
    "--algorithm hill_climbing --steps 100",
    "--algorithm greedy --steps 10",
    "--algorithm tabu_greedy --steps 10 --hparams '{\"n_tabu\":5}'",
    "--algorithm simulated_annealing --steps 100 --hparams '{\"t0\":20}'",
    "--algorithm csa --steps 100",
    "--algorithm cmaes --steps 100",
]

import pexpect

child = pexpect.spawn(f"sshpass -p {PASSWORD} \
                      ssh -tt {USER}@chome.metz.supelec.fr \
                      srun -p cpu_tp -N 1 -n 32 --pty bash",
                      timeout=5*60)

prompt = f"{USER}@kyle[0-9]+:"

for i in range(len(runs)):
    occupied = child.expect([prompt, "srun: job [1-9]+ queued and waiting for resources"])

    if occupied == 0:
        cmd = f"python3 -m optimizer.main {runs[i]} --log threads_run_{i}.log"
        child.sendline(cmd)
        running = 1
        while running == 1:
            running = child.expect(['Run finished', '\n'])
            output = child.before.decode()
            print(output)
        output = child.after.decode()
        print(output)

    elif occupied == 1:
        raise Exception("Error: no resource available. Try running `mysrun` and `scancel` on the remote machine.")
    
child.close()

In [ ]:
# make get_logs
!mkdir -p tmp/
!sshpass -p {PASSWORD} \
    scp '{USER}@chome.metz.supelec.fr:/usr/users/{GROUP}/{USER}/threads_run_*.log' ./tmp/

### Convert logs to data

### Plot graphs

**TODO:**

[ ] solution cost vs. evaluation no. (for many nthreads)

[ ] solution energy consumption vs. evaluation no. (for many nthreads)

[ ] speedup with 16 vs 32 threads (vs. evaluation no.)

[ ] compare speedup with energy consumption increase: in the same graph, plot these curves:

* speedup 16-32 threads vs evaluation no.

* energy consumption difference 16-32 threads vs. evaluation no.